In [ ]:
import random
import numpy as np

class Resource:
    def __init__(self, id, capacity):
        self.id = id
        self.capacity = capacity

class Task:
    def __init__(self, id, requirements):
        self.id = id
        self.requirements = requirements

class ResourceAllocationGeneticAlgorithm:
    def __init__(self, resources, tasks, population_size, num_generations, mutation_rate):
        self.resources = resources
        self.tasks = tasks
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate

    def create_individual(self):
        individual = np.zeros(len(self.tasks))
        for i in range(len(self.tasks)):
            requirements = self.tasks[i].requirements
            resource_id = random.choice(range(len(self.resources)))
            if requirements <= self.resources[resource_id].capacity:
                individual[i] = resource_id
        return individual

    def create_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.create_individual()
            population.append(individual)
        return population

    def calculate_fitness(self, individual):
        total_cost = 0
        for i in range(len(individual)):
            resource_id = int(individual[i])
            if resource_id >= len(self.resources):
                total_cost += float('inf')
            else:
                requirements = self.tasks[i].requirements
                total_cost += self.resources[resource_id].capacity - requirements
        return -total_cost  # Higher fitness for lower cost

    def selection(self, population):
        fitness_scores = [self.calculate_fitness(individual) for individual in population]
        total_fitness = sum(fitness_scores)
        probabilities = [fitness / total_fitness for fitness in fitness_scores]
        selected_indices = random.choices(range(len(population)), probabilities, k=2)
        return population[selected_indices[0]], population[selected_indices[1]]

    def crossover(self, parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2

    def mutation(self, individual):
        for i in range(len(individual)):
            if random.random() < self.mutation_rate:
                resource_id = random.choice(range(len(self.resources)))
                individual[i] = resource_id
        return individual

    def evolve_population(self, population):
        new_population = []
        while len(new_population) < self.population_size:
            parent1, parent2 = self.selection(population)
            child1, child2 = self.crossover(parent1, parent2)
            child1 = self.mutation(child1)
            child2 = self.mutation(child2)
            new_population.append(child1)
            new_population.append(child2)
        return new_population

    def find_best_solution(self):
        population = self.create_population()
        best_solution = None
        best_fitness = float('-inf')
        for _ in range(self.num_generations):
            population = self.evolve_population(population)
            fitness_scores = [self.calculate_fitness(individual) for individual in population]
            max_fitness = max(fitness_scores)
            if max_fitness > best_fitness:
                best_fitness = max_fitness
                best_solution = population[fitness_scores.index(max_fitness)]
        return best_solution


# Main program
resources = [
    Resource(1, 10),
    Resource(2, 15),
    Resource(3, 12)
]

tasks = [
    Task(1, 8),
    Task(2, 10),
    Task(3, 6),
    Task(4, 12),
    Task(5, 5)
]

population_size = 100
num_generations = 500
mutation_rate = 0.02

ga = ResourceAllocationGeneticAlgorithm(resources, tasks, population_size, num_generations, mutation_rate)
best_solution = ga.find_best_solution()

print("Best solution:", best_solution)



Best solution: [0. 0. 2. 0. 0.]
